In [1]:
cd ..

C:\Users\shank\Desktop\AUMC\colab\modys-video


# Experiment 1.1 Explore models and approaches for deep learning
Finetune models on left leg amplitude prediction with lying videos as input.
For now this is just to quickly test the settings that Shankara got out of previous experiments, i.e.:

* Cutoff first 50 frames
* Interpolate when likelihood is below 0.7
* Use opposite bodypart if all likelihood is below 0.7 for a bodypart
* Use a standard scaler to scale
    

We don't really know whether this is the best set of hyperparameters. We can further experiment with the following hyperparameters:
* Use a StandardScaler
* Include likelihood
* Use different model architectures
* Use interpolation
* write one function that takes in parameters

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import tensorflow.keras as keras


from src.helpers import read_scores
from src.data_generators import RawDataGenerator
from src.data_selection import MultipleScoreSelector
from src.ai_func import cross_validation_generator
from src.settings import LYING_VIDEOS_DATA_FOLDER, SITTING_VIDEOS_DATA_FOLDER, DATA_FOLDER


from keras.models import Sequential
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Lambda
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.advanced_activations import LeakyReLU

%load_ext autoreload
%autoreload 2

In [3]:
scores_df = read_scores(DATA_FOLDER / 'data_clinical_scoring.xlsx')

## Definitions

In [4]:
SCORES_TO_USE = ['D_LLP_R_tA_pscore']
SCORER_TO_USE = 1
SCORERS_TO_USE = [1, 2, 3]

n_outputs = len(SCORES_TO_USE)

## Pipeline for training a deep neural network

### Define model architecture (here: simple CNN)

In [5]:
def get_model(n_timesteps, n_features, n_outputs):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=32, kernel_size=5, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dense(n_outputs))
    return model


def get_model_old():
    # simple CNN
    input_layer = keras.layers.Input(shape=(n_timesteps,n_features))
    norm_layer = keras.layers.BatchNormalization()(input_layer)
    cnn_layer = keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu')(norm_layer)
    cnn_layer = keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu')(cnn_layer)
    cnn_layer = keras.layers.MaxPooling1D(pool_size=2)(cnn_layer)
    cnn_layer = keras.layers.Dropout(0.7)(cnn_layer)
    cnn_layer = keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu')(cnn_layer)
    cnn_layer = keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu')(cnn_layer)
    cnn_layer = keras.layers.MaxPooling1D(pool_size=2)(cnn_layer)
    cnn_layer = keras.layers.Dropout(0.7)(cnn_layer)
    cnn_layer = keras.layers.Flatten()(cnn_layer)
    cnn_layer = keras.layers.Dense(100)(cnn_layer)
    output_layer = keras.layers.Dense(n_outputs)(cnn_layer)

    return keras.Model(inputs=input_layer, outputs=output_layer)

## Train network

In [7]:
def train_model(train_generator, n_timesteps, n_features, n_outputs):
    model = get_model(n_timesteps, n_features, n_outputs)
    model.compile(loss='mae', optimizer=keras.optimizers.Adam())
    model.fit(train_generator, epochs=30)
    return model

def train_cross_val(cross_val):
    y_pred = []
    y_test = []
    splits = []
    for i_split, (train_scores, test_scores) in enumerate(cross_val):
        print(f'Fitting for 5-fold split {i_split}')
        train_generator = RawDataGenerator(train_scores, **data_generation_params)
        scaler = train_generator.get_scaler()
        data_generation_params['scaler'] = scaler
        test_generator = RawDataGenerator(test_scores, **data_generation_params)
        X, y = train_generator.__getitem__(0)
        n_timesteps, n_features = (X.shape[1], X.shape[2])
        model = train_model(train_generator, n_timesteps, n_features, n_outputs)
        y_pred.append(model.predict(test_generator))
        y_test.append(test_scores)
        splits.extend([i_split]*len(test_scores))
        break # This results in training only for one of the 5 folds
    y_pred = np.vstack(y_pred)
    y_test = pd.concat(y_test)
    return y_test, y_pred, splits

def evaluate(y_test, y_pred):
    results = []
    for i_score, column in enumerate(y_test):
        mae = mean_absolute_error(y_test.iloc[:, i_score], y_pred[:, i_score])
        results.append({'score': column, 'mae': mae})
    return pd.DataFrame(results)

## Test all combinations

In [8]:
results_df = pd.DataFrame(columns=['scorer', 'scores', 'split', 'y', 'pred_y'])

for scorer in SCORERS_TO_USE:
    for cutoff in [0,50]:
        for threshold in [0, 0.7]:
            for scaler in [None, StandardScaler()]:
                for bodyparts in [None, ['ankle1', 'knee1', 'hip1','ankle2', 'knee2', 'hip2']]:
                    data_generation_params = {
                        'videos_folder': LYING_VIDEOS_DATA_FOLDER,
                        'cutoff': cutoff,
                        'interpolation_threshold': threshold,
                        'batch_size': 1, 
                        'scaler': scaler,
                        'bodyparts': bodyparts
                    }  
                    print('params:',data_generation_params)
                    print(f'Training model for scorer {scorer}')
                    selector = MultipleScoreSelector(scores_to_use=SCORES_TO_USE, scorer_to_use=scorer)
                    selected_data = selector.transform(scores_df)
                    cross_val = cross_validation_generator(selected_data)
                    y_test, y_pred, splits = train_cross_val(cross_val)
                    for y_item, y_pred_item, split in zip(y_test.values, y_pred, splits):
                        results_df = results_df.append({'scorer':scorer, 
                                                        'scores': ', '.join(SCORES_TO_USE), 
                                                        'split': split, 
                                                        'y':y_item[0], 
                                                        'pred_y': y_pred_item[0], 
                                                        'videos_folder': data_generation_params['videos_folder'],
                                                        'cutoff': data_generation_params['cutoff'],
                                                        'interpolation_threshold': data_generation_params['interpolation_threshold'],
                                                        'batch_size': data_generation_params['batch_size'],
                                                        'scaler': data_generation_params['scaler'],
                                                        'bodyparts': data_generation_params['bodyparts']}, ignore_index=True)

params: {'videos_folder': WindowsPath('C:/Users/shank/Desktop/AUMC/colab/modys-video/data/data_lying_052929'), 'cutoff': 0, 'interpolation_threshold': 0, 'batch_size': 1, 'scaler': None, 'bodyparts': None}
Training model for scorer 1


C:\Users\shank\Desktop\AUMC\colab\modys-video\src\data_selection.py:24: UserWarning: Dropping rows ['096'] with missing video
  warnings.warn(f'Dropping rows {missing_ids} with missing video')
C:\Users\shank\Desktop\AUMC\colab\modys-video\src\data_selection.py:34: UserWarning: Dropping rows ['090' '058' '048'] with missing score
  warnings.warn(f'Dropping rows {only_na.index.values} with missing score')


Fitting for 5-fold split 0
Epoch 1/30
49/49 [==============================] - 3s 39ms/step - loss: 90.5692
Epoch 2/30
49/49 [==============================] - 2s 37ms/step - loss: 11.3393
Epoch 3/30
49/49 [==============================] - 2s 38ms/step - loss: 0.5762
Epoch 4/30
49/49 [==============================] - 2s 38ms/step - loss: 0.2400
Epoch 5/30
49/49 [==============================] - 2s 37ms/step - loss: 0.2601
Epoch 6/30
49/49 [==============================] - 2s 38ms/step - loss: 0.2474
Epoch 7/30
49/49 [==============================] - 2s 38ms/step - loss: 0.2403
Epoch 8/30
49/49 [==============================] - 2s 39ms/step - loss: 0.2386
Epoch 9/30
49/49 [==============================] - 2s 38ms/step - loss: 0.2388
Epoch 10/30
49/49 [==============================] - 2s 38ms/step - loss: 0.2449
Epoch 11/30
49/49 [==============================] - 2s 38ms/step - loss: 0.2524
Epoch 12/30
49/49 [==============================] - 2s 37ms/step - loss: 0.2464
Epoch 13

C:\Users\shank\Desktop\AUMC\colab\modys-video\src\data_selection.py:24: UserWarning: Dropping rows ['096'] with missing video
  warnings.warn(f'Dropping rows {missing_ids} with missing video')
C:\Users\shank\Desktop\AUMC\colab\modys-video\src\data_selection.py:34: UserWarning: Dropping rows ['090' '058' '048'] with missing score
  warnings.warn(f'Dropping rows {only_na.index.values} with missing score')


Fitting for 5-fold split 0
Epoch 1/30
49/49 [==============================] - 3s 38ms/step - loss: 69.5458
Epoch 2/30
49/49 [==============================] - 2s 40ms/step - loss: 14.0388
Epoch 3/30
49/49 [==============================] - 2s 40ms/step - loss: 4.2007
Epoch 4/30
49/49 [==============================] - 2s 38ms/step - loss: 1.7939
Epoch 5/30
49/49 [==============================] - 2s 37ms/step - loss: 1.5512
Epoch 6/30
49/49 [==============================] - 2s 39ms/step - loss: 0.9354
Epoch 7/30
49/49 [==============================] - 2s 37ms/step - loss: 0.4855
Epoch 8/30
49/49 [==============================] - 2s 37ms/step - loss: 0.5754
Epoch 9/30
49/49 [==============================] - 2s 37ms/step - loss: 0.3743
Epoch 10/30
49/49 [==============================] - 2s 37ms/step - loss: 0.2889
Epoch 11/30
49/49 [==============================] - 2s 38ms/step - loss: 0.2697: 0s - loss: 0.
Epoch 12/30
49/49 [==============================] - 2s 42ms/step - loss: 

...

Fitting for 5-fold split 0
Epoch 1/30
36/36 [==============================] - 2s 37ms/step - loss: 97.8545
Epoch 2/30
36/36 [==============================] - 1s 39ms/step - loss: 11.6054
Epoch 3/30
36/36 [==============================] - 1s 40ms/step - loss: 0.6835
Epoch 4/30
36/36 [==============================] - 1s 38ms/step - loss: 0.3051
Epoch 5/30
36/36 [==============================] - 1s 40ms/step - loss: 0.2819
Epoch 6/30
36/36 [==============================] - 1s 37ms/step - loss: 0.2328
Epoch 7/30
36/36 [==============================] - 2s 45ms/step - loss: 0.2508
Epoch 8/30
36/36 [==============================] - 1s 40ms/step - loss: 0.2766
Epoch 9/30
36/36 [==============================] - 1s 37ms/step - loss: 0.2487
Epoch 10/30
36/36 [==============================] - 2s 45ms/step - loss: 0.2531
Epoch 11/30
36/36 [==============================] - 2s 49ms/step - loss: 0.2718
Epoch 12/30
36/36 [==============================] - 2s 49ms/step - loss: 0.2690
Epoch 13

C:\Users\shank\Desktop\AUMC\colab\modys-video\src\data_selection.py:34: UserWarning: Dropping rows [] with missing score
  warnings.warn(f'Dropping rows {only_na.index.values} with missing score')


Fitting for 5-fold split 0
Epoch 1/30
36/36 [==============================] - 2s 36ms/step - loss: 412.7529
Epoch 2/30
36/36 [==============================] - 1s 36ms/step - loss: 150.6303
Epoch 3/30
36/36 [==============================] - 1s 36ms/step - loss: 71.9697
Epoch 4/30
36/36 [==============================] - 1s 36ms/step - loss: 34.0166
Epoch 5/30
36/36 [==============================] - 1s 36ms/step - loss: 10.6863
Epoch 6/30
36/36 [==============================] - 1s 36ms/step - loss: 4.0677
Epoch 7/30
36/36 [==============================] - 1s 36ms/step - loss: 1.7662
Epoch 8/30
36/36 [==============================] - 1s 36ms/step - loss: 1.5031
Epoch 9/30
36/36 [==============================] - 1s 36ms/step - loss: 2.1622
Epoch 10/30
36/36 [==============================] - 1s 37ms/step - loss: 0.6753
Epoch 11/30
36/36 [==============================] - 1s 36ms/step - loss: 0.4208
Epoch 12/30
36/36 [==============================] - 1s 36ms/step - loss: 0.5464
Epo

C:\Users\shank\Desktop\AUMC\colab\modys-video\src\data_selection.py:34: UserWarning: Dropping rows [] with missing score
  warnings.warn(f'Dropping rows {only_na.index.values} with missing score')


Fitting for 5-fold split 0
Epoch 1/30
36/36 [==============================] - 2s 36ms/step - loss: 310.4989
Epoch 2/30
36/36 [==============================] - 1s 38ms/step - loss: 165.8254
Epoch 3/30
36/36 [==============================] - 1s 36ms/step - loss: 298.2274
Epoch 4/30
36/36 [==============================] - 1s 37ms/step - loss: 34.3410
Epoch 5/30
36/36 [==============================] - 1s 37ms/step - loss: 12.2117
Epoch 6/30
36/36 [==============================] - 1s 37ms/step - loss: 5.5580
Epoch 7/30
36/36 [==============================] - 1s 37ms/step - loss: 0.9180
Epoch 8/30
36/36 [==============================] - 1s 37ms/step - loss: 2.2092
Epoch 9/30
36/36 [==============================] - 1s 36ms/step - loss: 0.6674
Epoch 10/30
36/36 [==============================] - 1s 37ms/step - loss: 1.3862
Epoch 11/30
36/36 [==============================] - 1s 37ms/step - loss: 0.5688
Epoch 12/30
36/36 [==============================] - 1s 37ms/step - loss: 0.7162
Ep

In [9]:
results_df['MAE'] = abs(results_df['y'] - results_df['pred_y'])

In [10]:
results_df

,scorer,scores,split,y,pred_y,batch_size,bodyparts,cutoff,interpolation_threshold,scaler,videos_folder,MAE
0,1,D_LLP_R_tA_pscore,0,0.50,0.513004,1.0,None,0.0,0.0,None,C:\Users\shank\Desktop\AUMC\colab\modys-video\...,0.013004
1,1,D_LLP_R_tA_pscore,0,0.50,0.513004,1.0,None,0.0,0.0,None,C:\Users\shank\Desktop\AUMC\colab\modys-video\...,0.013004
2,1,D_LLP_R_tA_pscore,0,0.75,0.513004,1.0,None,0.0,0.0,None,C:\Users\shank\Desktop\AUMC\colab\modys-video\...,0.236996
3,1,D_LLP_R_tA_pscore,0,0.50,0.513004,1.0,None,0.0,0.0,None,C:\Users\shank\Desktop\AUMC\colab\modys-video\...,0.013004
4,1,D_LLP_R_tA_pscore,0,0.50,0.513004,1.0,None,0.0,0.0,None,C:\Users\shank\Desktop\AUMC\colab\modys-video\...,0.013004
...,...,...,...,...,...,...,...,...,...,...,...,...
539,3,D_LLP_R_tA_pscore,0,0.75,0.293107,1.0,"[ankle1, knee1, hip1, ankle2, knee2, hip2]",50.0,0.7,StandardScaler(),C:\Users\shank\Desktop\AUMC\colab\modys-video\...,0.456893
540,3,D_LLP_R_tA_pscore,0,0.25,-0.025361,1.0,"[ankle1, knee1, hip1, ankle2, knee2, hip2]",50.0,0.7,StandardScaler(),C:\Users\shank\Desktop\AUMC\colab\modys-video\...,0.275361
541,3,D_LLP_R_tA_pscore,0,0.50,0.438473,1.0,"[ankle1, knee1, hip1, ankle2, knee2, hip2]",50.0,0.7,StandardScaler(),C:\Users\shank\Desktop\AUMC\colab\modys-video\...,0.061527
542,3,D_LLP_R_tA_pscore,0,0.50,0.293107,1.0,"[ankle1, knee1, hip1, ankle2, knee2, hip2]",50.0,0.7,StandardScaler(),C:\Users\shank\Desktop\AUMC\colab\modys-video\...,0.206893


In [9]:
results_df.to_excel('results/experiment-1.1-results-validation.xlsx', index=False)

In [24]:
for queryset in [['scorer == 1', 'scorer == 2', 'scorer == 3'], ['cutoff == 50', 'cutoff == 0'], 
                 ['interpolation_threshold==0.7', 'interpolation_threshold==0'], ['scaler!=scaler', 'scaler==scaler'], 
                ['bodyparts!=bodyparts', 'bodyparts==bodyparts']]:
    for query in queryset:
        print(query)
        print('Mean:',results_df.query(query)['MAE'].mean())
        print('Median:',results_df.query(query)['MAE'].median())
        print()

scorer == 1
Mean: 0.3539497444334512
Median: 0.07257473468780518

scorer == 2
Mean: 0.3760270569473505
Median: 0.19471707940101624

scorer == 3
Mean: 0.48476964664878325
Median: 0.2674099802970886

cutoff == 50
Mean: 0.3582480322186123
Median: 0.17742526531219482

cutoff == 0
Mean: 0.4408896601663026
Median: 0.21186068654060364

interpolation_threshold==0.7
Mean: 0.3234142663902329
Median: 0.20683592557907104

interpolation_threshold==0
Mean: 0.475723425994682
Median: 0.1950056552886963

scaler!=scaler
Mean: 0.15026506096782052
Median: 0.1664780080318451

scaler==scaler
Mean: 0.6488726314170944
Median: 0.22859150171279907

bodyparts!=bodyparts
Mean: 0.3532122304827413
Median: 0.20204156637191772

bodyparts==bodyparts
Mean: 0.4459254619021736
Median: 0.20658156275749207



In [25]:
for scorer in ['scorer == 1', 'scorer == 2', 'scorer == 3']:
    for cutoff in ['cutoff == 50', 'cutoff == 0']:
        for interpolation in ['interpolation_threshold==0.7', 'interpolation_threshold==0']:
            for scaler in ['scaler!=scaler', 'scaler==scaler']:
                for bodyparts in ['bodyparts!=bodyparts', 'bodyparts==bodyparts']:
                    print(' and '.join([scorer, cutoff, interpolation, scaler, bodyparts]))
                    print('Mean:', results_df.query(' and '.join([scorer, cutoff, interpolation, scaler, bodyparts]))['MAE'].mean())
                    print('Median:', results_df.query(' and '.join([scorer, cutoff, interpolation, scaler, bodyparts]))['MAE'].median())
                    print()

scorer == 1 and cutoff == 50 and interpolation_threshold==0.7 and scaler!=scaler and bodyparts!=bodyparts
Mean: 0.11277723770875198
Median: 0.04322081804275513

scorer == 1 and cutoff == 50 and interpolation_threshold==0.7 and scaler!=scaler and bodyparts==bodyparts
Mean: 0.10475539244138278
Median: 0.022364020347595215

scorer == 1 and cutoff == 50 and interpolation_threshold==0.7 and scaler==scaler and bodyparts!=bodyparts
Mean: 0.14517642442996687
Median: 0.1959235668182373

scorer == 1 and cutoff == 50 and interpolation_threshold==0.7 and scaler==scaler and bodyparts==bodyparts
Mean: 0.6888237607020599
Median: 0.16883152723312378

scorer == 1 and cutoff == 50 and interpolation_threshold==0 and scaler!=scaler and bodyparts!=bodyparts
Mean: 0.10684568148392898
Median: 0.027798771858215332

scorer == 1 and cutoff == 50 and interpolation_threshold==0 and scaler!=scaler and bodyparts==bodyparts
Mean: 0.12406720564915584
Median: 0.07257473468780518

scorer == 1 and cutoff == 50 and inter

In [11]:
results_df.query('cutoff == 50 and scorer==1 and interpolation_threshold==0.7 and scaler!=scaler and bodyparts!=bodyparts')['MAE'].mean()

0.11277723770875198

## Run Once

In [16]:
data_generation_params = {
    'videos_folder': LYING_VIDEOS_DATA_FOLDER,
    'cutoff': 50,
    'interpolation_threshold': 0.7,
    'batch_size': 1, 
    'scaler': None,
    'bodyparts': None
} 

In [17]:
print(f'Training model for scorer {SCORER_TO_USE}')
selector = MultipleScoreSelector(scores_to_use=SCORES_TO_USE, scorer_to_use=SCORER_TO_USE)
selected_data = selector.transform(scores_df)
cross_val = cross_validation_generator(selected_data)
y_test, y_pred, splits = train_cross_val(cross_val)

Training model for scorer 1


C:\Users\shank\Desktop\AUMC\colab\modys-video\src\data_selection.py:24: UserWarning: Dropping rows ['096'] with missing video
  warnings.warn(f'Dropping rows {missing_ids} with missing video')
C:\Users\shank\Desktop\AUMC\colab\modys-video\src\data_selection.py:34: UserWarning: Dropping rows ['090' '058' '048'] with missing score
  warnings.warn(f'Dropping rows {only_na.index.values} with missing score')


Fitting for 5-fold split 0
Epoch 1/30
49/49 [==============================] - 3s 39ms/step - loss: 78.9575
Epoch 2/30
49/49 [==============================] - 2s 37ms/step - loss: 18.4503
Epoch 3/30
49/49 [==============================] - 2s 42ms/step - loss: 5.0366
Epoch 4/30
49/49 [==============================] - 2s 38ms/step - loss: 4.7940
Epoch 5/30
49/49 [==============================] - 2s 38ms/step - loss: 0.7909
Epoch 6/30
49/49 [==============================] - 2s 38ms/step - loss: 0.3998
Epoch 7/30
49/49 [==============================] - 2s 37ms/step - loss: 0.2407
Epoch 8/30
49/49 [==============================] - 2s 39ms/step - loss: 0.2376
Epoch 9/30
49/49 [==============================] - 2s 40ms/step - loss: 0.2300
Epoch 10/30
49/49 [==============================] - 2s 44ms/step - loss: 0.2292
Epoch 11/30
49/49 [==============================] - 2s 48ms/step - loss: 0.2289
Epoch 12/30
49/49 [==============================] - 2s 50ms/step - loss: 0.2306
Epoch 13

In [18]:
print(evaluate(y_test, y_pred))

               score       mae
0  D_LLP_R_tA_pscore  0.101455
